<a href="https://colab.research.google.com/github/cccaaannn/deep_learning_colab/blob/master/keras/utilities/convert_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Check the gpu**

In [ ]:
!nvidia-smi

**Download and unzip dataset**

In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
!unzip kagglecatsanddogs_3367a.zip

**Install extra libs**

In [ ]:
!pip install imagepreprocessing

**Imports has to be tensorflow.keras**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# keras and tf
import tensorflow as tf
from tensorflow import keras

# models
from tensorflow.keras.models import Sequential

# backend
from tensorflow.keras import optimizers, metrics, models
import tensorflow.keras.backend as K

# layers
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization

# optimizers
from tensorflow.keras.optimizers import SGD

# training
from tensorflow.keras.callbacks import ModelCheckpoint

# other libs
import numpy as np
import math
import pickle
import os

In [ ]:
# imagepreprocessing
from  imagepreprocessing.keras_functions import create_training_data_keras, make_prediction_from_array_keras
from  imagepreprocessing.utilities import create_confusion_matrix, train_test_split

**Create training data**

In [ ]:
x, y, test_x, test_y  = create_training_data_keras("/content/PetImages", image_size=224, percent_to_use=0.5, validation_split=0.2, grayscale=True)

**Create model**

In [ ]:
# model 7 with batch norm
model = Sequential()
model.add(Conv2D(16, (3,3), input_shape=x.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(2, activation='softmax'))

**compile model**

In [ ]:
opt = SGD(lr=0.001, momentum=0.9)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

checkpointer

In [ ]:
checkpointer = ModelCheckpoint(filepath= "/content/model", verbose=1, monitor='val_loss', save_best_only=True)

**Fit**

In [ ]:
model.fit(x,y, validation_split=0.2, epochs=15, callbacks=[checkpointer])

**Evaluate**

In [ ]:
class_names = ["cat", "dog"]

# make prediction
predictions = make_prediction_from_array_keras(test_x, model, print_output=False)

# create confusion matrix
cnf_matrix = create_confusion_matrix(predictions, test_y, class_names=class_names, one_hot=True)


accuracy = (cnf_matrix[0][0]+cnf_matrix[1][1]) / sum(sum(cnf_matrix))

prescision_dog = cnf_matrix[1][1]/(cnf_matrix[1][1]+cnf_matrix[0][1])
recall_dog = cnf_matrix[1][1]/(cnf_matrix[1][1]+cnf_matrix[1][0])

prescision_cat = cnf_matrix[0][0]/(cnf_matrix[0][0]+cnf_matrix[1][0])
recall_cat = cnf_matrix[0][0]/(cnf_matrix[0][0]+cnf_matrix[0][1])


print("accuracy {}".format(accuracy))

print("prescision_dog {} recall_dog {}".format(prescision_dog,recall_dog))

print("prescision_cat {} recall_cat {}".format(prescision_cat,recall_cat))


**convert to tflite**

In [ ]:

# model = tf.keras.models.load_model('/content/model')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

# show type
print("type of the model is:", type(model))
print("type of the converted model is:", type(tflite_float_model))

# Show model size in KBs.
float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

# save
open("model.tflite", "wb").write(tflite_float_model)